In [42]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

tf.__version__

'2.10.0'

ZOO classification
Data list
동물 이름 animal name: (deleted)
털 hair Boolean
깃털 feathers Boolean
알 eggs Boolean
우유 milk Boolean
날 수있는지 airborne Boolean
수중 생물 aquatic Boolean
포식자 predator Boolean
이빨이 있는지 toothed Boolean
척추 동물 backbone Boolean
호흡 방법 breathes Boolean
독 venomous Boolean
물갈퀴 fins Boolean
다리 legs Numeric (set of values: {0",2,4,5,6,8})
꼬리 tail Boolean
사육 가능한 지 domestic Boolean
고양이 크기인지 catsize Boolean
동물 타입 type Numeric (integer values in range [0",6])

In [43]:
xy = np.loadtxt('data-04-zoo.csv',
                delimiter=',',
                dtype=np.int32)
x_train = xy[0:-10, 0:-1]
y_train = xy[0:-10, [-1]]


x_train = tf.cast(x_train, tf.float32)

x_test = xy[-10:, 0:-1]
y_test = xy[-10:, [-1]]

x_test = tf.cast(x_test, tf.float32)

nb_classes = 7

y_train = tf.one_hot(list(y_train), nb_classes)
y_train = tf.reshape(y_train, [-1, nb_classes])
print(y_train[15])

y_test = tf.one_hot(list(y_test), nb_classes) # one hot 으로 변환
print(f"before reshape:{y_test.shape}")
y_test = tf.reshape(y_test, [-1, nb_classes])

tf.Tensor([0. 0. 0. 0. 0. 0. 1.], shape=(7,), dtype=float32)
before reshape:(10, 1, 7)


In [44]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))

W = tf.Variable(tf.random.normal([16,nb_classes],0,1), name='weight')
b = tf.Variable(tf.random.normal([nb_classes],0,1), name='bias')

print(W.shape, b.shape)
print(len(x_train))

(16, 7) (7,)
91


In [45]:
def logistic_regression(features): # hypothesis_softmax
    return tf.nn.softmax(tf.matmul(features, W) + b)

print(logistic_regression(x_train))

tf.Tensor(
[[4.10938100e-07 1.39875337e-03 1.85227429e-04 2.16101602e-04
  1.86940633e-06 9.98000562e-01 1.97088782e-04]
 [1.59132935e-06 5.75020676e-04 2.43113172e-04 1.00991529e-04
  7.00534247e-06 9.98667598e-01 4.04730410e-04]
 [7.83748031e-02 8.18328142e-01 1.52283655e-02 2.52695531e-02
  8.26386642e-03 1.44316591e-02 4.01035249e-02]
 [4.10938100e-07 1.39875337e-03 1.85227429e-04 2.16101602e-04
  1.86940633e-06 9.98000562e-01 1.97088782e-04]
 [8.65881873e-07 1.41055486e-03 1.71979947e-04 4.03089325e-05
  2.61753598e-06 9.98294532e-01 7.91173661e-05]
 [1.59132935e-06 5.75020676e-04 2.43113172e-04 1.00991529e-04
  7.00534247e-06 9.98667598e-01 4.04730410e-04]
 [4.32815284e-07 1.26512023e-03 3.18771927e-04 1.08118380e-04
  3.33464850e-05 9.97328877e-01 9.45373205e-04]
 [2.66861413e-02 4.99959111e-01 1.92274172e-02 4.61701304e-02
  7.17142597e-02 9.82112810e-03 3.26421678e-01]
 [7.83748031e-02 8.18328142e-01 1.52283655e-02 2.52695531e-02
  8.26386642e-03 1.44316591e-02 4.01035249e-02]

In [46]:
def loss_fn(hypothesis, labels):
    #loss = -tf.reduce_mean(tf.reduce_sum(labels * tf.math.log(hypothesis) +(1 - labels) * tf.math.log(1 - hypothesis), axis=1))
    loss = tf.keras.losses.categorical_crossentropy(labels, hypothesis)
    return loss

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

In [47]:
epochs = 500

for step in range(epochs):
    for features, labels in dataset:
        with tf.GradientTape() as tape:
            pred = logistic_regression(features)
            loss_value = loss_fn(pred,labels)
        grads = tape.gradient(loss_value, [W,b])
        optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]))
        if step % 10 == 0:
            print("Iter: {}, Loss: {}".format(step, loss_fn(logistic_regression(features),labels)))


Iter: 0, Loss: [2.338424   0.8476174  3.12873    2.338424   1.4619673  0.8476174
 2.382046   2.4702325  3.12873    5.189381   1.4619673  1.6121482
 3.12873    2.608707   2.9780138  3.313283   0.7789089  0.8476174
 4.674965   0.10128376 1.6121482  3.0445414  0.8476174  3.4949574
 5.4957767  6.704721   6.5139203  3.0134673  0.8476174  3.5475345
 4.710898   2.382046   1.3864442  1.7590175  2.245143   4.043834
 2.1817484  0.7789089  3.12873    4.973328   6.2519298  1.7950922
 4.6310463  1.8421462  1.4619673  1.4619673  3.313283   1.4619673
 0.53194785 3.2435904  1.4619673  6.2519298  6.5895133  1.4833516
 3.2435904  0.8476174  4.9270105  1.6121482  5.1592426  1.8421462
 4.674965   3.12873    1.5330995  0.39697537 1.4619673  2.382046
 0.10128376 1.4619673  3.2546992  1.4619673  2.382046   3.6032166
 4.5061197  2.245143   0.93336207 0.76671034 3.307961   2.4187446
 1.7590175  1.7590175  2.1845906  1.5341132  2.245143   1.8421462
 1.6156437  3.1380134  6.179671   4.896822   5.4957767  6.31231

In [61]:
def accuracy_fn(hypothesis, labels):
    hypothesis = tf.argmax(hypothesis, 1)
    predicted = tf.cast(hypothesis, dtype=tf.float32)
    print(predicted)

    labels = tf.argmax(labels, 1)
    labels = tf.cast(labels, dtype=tf.float32)
    print(labels)

    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))

    return accuracy

In [63]:
tesr_acc = accuracy_fn(logistic_regression(x_test), y_test)
print("Testset Accuracy: {:.4f}".format(tesr_acc))

tf.Tensor([2. 3. 0. 0. 1. 0. 5. 0. 6. 1.], shape=(10,), dtype=float32)
tf.Tensor([2. 3. 0. 0. 1. 0. 5. 0. 6. 1.], shape=(10,), dtype=float32)
Testset Accuracy: 1.0000
